# <center>Organizaci&oacute;n de Datos</center>
## <center>Trabajo Práctico 2 </center>

## <center>Grupo 21</center>

#### Integrantes: Francisco Juárez, Lucas Aldazabal, Valentin Schneider, Gonzalo Rizzo Ehrenbock

# <center> Procesamiento del Lenguaje Natural <center/> 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

sns.set_theme(style="whitegrid")

In [2]:
#Import dataframe
ds_propiedades = pd.read_csv('./ds_propiedades_total_clasificado.csv')
ds_descripciones = pd.read_csv('./properati_argentina_2021_decrip.csv')
ds_propiedades.head()

,id,latitud,longitud,place_l3,property_type,property_rooms,property_bedrooms,property_surface_total,property_surface_covered,property_price,property_title,pxm2,tipo_precio
0,F5IvmD1uzzdIR7b96ozt4g==,-34.558855,-58.452458,Nuñez,Departamento,4.0,3.0,160.0,146.0,460000.0,Venta: Hermoso piso de categoría en excelente ...,2875.000000,alto
1,IiT/P7WfYDcqbhu7tf5V4Q==,-34.582403,-58.443841,Colegiales,Departamento,2.0,1.0,50.0,46.0,148000.0,Departamento - Colegiales,2960.000000,alto
2,6XXcElx/4OwCcH4NI06Oiw==,-34.605825,-58.422214,Almagro,Departamento,3.0,2.0,47.0,47.0,89000.0,Departamento en venta tres ambientes- Almagro,1893.617021,medio
3,8R7hSzqWFk9rX4wBXxfORA==,-34.561971,-58.455670,Belgrano,Departamento,2.0,1.0,52.0,48.0,127000.0,Departamento - Belgrano,2442.307692,medio
4,UAKYqQx9s0MfIdLzrJv8Ig==,-34.606738,-58.389426,Tribunales,Departamento,3.0,2.0,76.0,75.0,129900.0,Lindísimo departamento antiguo en excelente es...,1709.210526,bajo


In [3]:
ds_descripciones = ds_descripciones[ds_descripciones['id'].isin(ds_propiedades['id'])]
ds_descripciones.head()

,id,property_description
0,BZCeiGkJr1WBUi6pKJQwJQ==,Corredor Responsable: MARIA ALEJANDRA GENOVEVA...
2,TUL813tXchVBlHUsfaA6DA==,Excelente PH en Núñez 4 amb!!!! Oportunidad !!...
3,gn55JPmcjftBV/at1a7fPg==,"UNICO PH DE 4 DORMITORIOS EN IMPECABLE ESTADO,..."
10,+9uGr8tPEUkKnpjOef+gUw==,SE VENDE EXCELENTE PH de 2 CÓMODOS AMBIENTES M...
11,uH5H+RfQM87Qtrb13+T4hA==,"Venta PH 4 ambientes Paternal<br><br>Cálido, ..."


## Exportamos el dataframe a un archivo de texto para poder pasarlo por ECmes

In [4]:
ds_descripciones_sin_id = ds_descripciones.drop(['id'], axis=1)

ds_descripciones_sin_id.to_csv('ds_descripciones.txt', index=False)

## Limpiamos el archivo de descripciones para que lo pueda leer Ecmes correctamente.

In [5]:
# dic_replace = {
#     'á': 'a',
#     'é': 'e',
#     'í': 'i',
#     'ó': 'o',
#     'ú': 'u',
#     'Á': 'A',
#     'É': 'E',
#     'Í': 'I',
#     'Ó': 'O',
#     'Ú': 'U',
#     'ñ': 'ni',
#     'Ñ': 'NI',
#     'ü': 'u',
#     'Ü': 'U',
#     '<br>': ' ',
#     '<br />': ' ',
#     '<br/>': ' ',
#     '<p>': ' ',
#     '</p>': ' ',
#     '<b>': ' ',
#     '</b>': ' ',
# }

In [6]:
# #for every line in ds_descripciones.txt replace the '(' and ')' with ',' and save it in ds_descripciones.txt line by line 
# with open('ds_descripciones.txt', 'r', encoding = 'utf8') as f_in, open('ds_descripciones_out.txt', 'w', encoding = 'utf8') as f_out:
#     for line in f_in:
#         for src, target in dic_replace.items():
#             line = line.replace(src, target)
        
       
#         #leave only alpha characters in the line, also spaces and line jumps
#         line = ''.join([c for c in line if c.isalpha() or c == ' ' or c == '\n' or c == '.' or c == ','])
 
        
   

#         # for src, target in dic_replace.items():
#         #     line = line.replace(src, target)
#         f_out.writelines(line)

In [7]:
ds_propiedades = ds_propiedades.join(ds_descripciones.set_index('id'), on='id')
ds_propiedades.head()

,id,latitud,longitud,place_l3,property_type,property_rooms,property_bedrooms,property_surface_total,property_surface_covered,property_price,property_title,pxm2,tipo_precio,property_description
0,F5IvmD1uzzdIR7b96ozt4g==,-34.558855,-58.452458,Nuñez,Departamento,4.0,3.0,160.0,146.0,460000.0,Venta: Hermoso piso de categoría en excelente ...,2875.000000,alto,Hermoso piso de 4 ambientes con dependencia y ...
1,IiT/P7WfYDcqbhu7tf5V4Q==,-34.582403,-58.443841,Colegiales,Departamento,2.0,1.0,50.0,46.0,148000.0,Departamento - Colegiales,2960.000000,alto,2 ambientes con balcón y vista totalmente abie...
2,6XXcElx/4OwCcH4NI06Oiw==,-34.605825,-58.422214,Almagro,Departamento,3.0,2.0,47.0,47.0,89000.0,Departamento en venta tres ambientes- Almagro,1893.617021,medio,**DIAZ VELEZ 4291 Y PRINGLES** Almagro.<br><br...
3,8R7hSzqWFk9rX4wBXxfORA==,-34.561971,-58.455670,Belgrano,Departamento,2.0,1.0,52.0,48.0,127000.0,Departamento - Belgrano,2442.307692,medio,IMPECABLE 2 AMBIENTES A ESTRENAR!<...
4,UAKYqQx9s0MfIdLzrJv8Ig==,-34.606738,-58.389426,Tribunales,Departamento,3.0,2.0,76.0,75.0,129900.0,Lindísimo departamento antiguo en excelente es...,1709.210526,bajo,"Excelente piso antiguo, lindísimo por sus tech..."


## Imputacion de nuevos parametros basados en los datos de Ecmes

#### Basandonos en la frecuencia de aparición de palabras dada por Ecmes y nuestro dominio del problema, sabemos que estos son aspectos muy influyentes.<br>
#### Procedemos a añadirlos al dataset para ampliarlo

#### Pileta

In [8]:
import re
regexp = re.compile(r'piscina|pileta/gi', re.IGNORECASE)

ds_propiedades['piscina'] = ds_propiedades['property_description'].apply(lambda x: 1 if regexp.search(x) else 0)

In [9]:
cantidad_pileta = ds_propiedades[ds_propiedades['piscina'] == 1].shape[0]
print(f"Vemos que la cantidad de propiedades con pileta es de {cantidad_pileta}")

Vemos que la cantidad de propiedades con pileta es de 9729


#### Garaje/Cochera

In [10]:
regexp  = re.compile(r'cochera|garage/gi', re.IGNORECASE)

ds_propiedades['cochera'] = ds_propiedades['property_description'].apply(lambda x: 1 if regexp.search(x) else 0)

In [11]:
cantidad_garaje = ds_propiedades[ds_propiedades['cochera'] == 1].shape[0]
print(f"Vemos que la cantidad de propiedades con Garaje es de {cantidad_garaje}")

Vemos que la cantidad de propiedades con Garaje es de 27031


#### Balcon

In [12]:
regexp  = re.compile(r'balc[oó]n|/gi', re.IGNORECASE)

ds_propiedades['balcon'] = ds_propiedades['property_description'].apply(lambda x: 1 if regexp.search(x) else 0)

In [13]:
cantidad_balcon = ds_propiedades[ds_propiedades['balcon'] == 1].shape[0]
print(f"Vemos que la cantidad de propiedades con Balcon es de {cantidad_balcon}")

Vemos que la cantidad de propiedades con Balcon es de 52839


#### Hidromasaje

In [14]:
regexp  = re.compile(r'hidromasaje|jacuzzi|spa/gi', re.IGNORECASE)

ds_propiedades['hidromasaje'] = ds_propiedades['property_description'].apply(lambda x: 1 if regexp.search(x) else 0)

In [15]:
cantidad_hidromasaje = ds_propiedades[ds_propiedades['hidromasaje'] == 1].shape[0]
print(f"Vemos que la cantidad de propiedades con Hidromasaje es de {cantidad_hidromasaje}")

Vemos que la cantidad de propiedades con Hidromasaje es de 4047


#### Jardín

In [16]:
regexp = re.compile(r'jardin|jardín/gi', re.IGNORECASE)

ds_propiedades['jardin'] = ds_propiedades['property_description'].apply(lambda x: 1 if regexp.search(x) else 0)

In [17]:
cantidad_jardin = ds_propiedades[ds_propiedades['jardin'] == 1].shape[0]
print(f"Vemos que la cantidad de propiedades con Jardin es de {cantidad_jardin}")

Vemos que la cantidad de propiedades con Jardin es de 2688


#### Gimnasio

In [18]:
regexp = re.compile(r'gimnasio|gym/gi', re.IGNORECASE)

ds_propiedades['gimnasio'] = ds_propiedades['property_description'].apply(lambda x: 1 if regexp.search(x) else 0)

In [19]:
cantidad_gimnasio = ds_propiedades[ds_propiedades['gimnasio'] == 1].shape[0]
print(f"Vemos que la cantidad de propiedades con Gimnasio es de {cantidad_gimnasio}")

Vemos que la cantidad de propiedades con Gimnasio es de 6601


#### A estrenar

In [20]:
regexp = re.compile(r'estrenar|/gi', re.IGNORECASE)

ds_propiedades['a_estrenar'] = ds_propiedades['property_description'].apply(lambda x: 1 if regexp.search(x) else 0)

In [21]:
cantidad_a_estrenar = ds_propiedades[ds_propiedades['a_estrenar'] == 1].shape[0]
print(f"Vemos que la cantidad de propiedades a estrenar es de {cantidad_a_estrenar}")

Vemos que la cantidad de propiedades a estrenar es de 8311


#### Sótano

In [22]:
regexp = re.compile(r'sotano|sótano/gi', re.IGNORECASE)

ds_propiedades['sotano'] = ds_propiedades['property_description'].apply(lambda x: 1 if regexp.search(x) else 0)

In [23]:
cantidad_sotano = ds_propiedades[ds_propiedades['sotano'] == 1].shape[0]
print(f"Vemos que la cantidad de propiedades con Sotano es de {cantidad_sotano}")

Vemos que la cantidad de propiedades con Sotano es de 83


#### Quincho

In [24]:
regexp = re.compile(r'quincho|/gi', re.IGNORECASE)

ds_propiedades['quincho'] = ds_propiedades['property_description'].apply(lambda x: 1 if regexp.search(x) else 0)

In [25]:
cantidad_quincho = ds_propiedades[ds_propiedades['quincho'] == 1].shape[0]
print(f"Vemos que la cantidad de propiedades con Quincho es de {cantidad_quincho}")

Vemos que la cantidad de propiedades con Quincho es de 3173


#### Sum

In [26]:
regexp = re.compile(r'(\bsum\b)|(s\.u\.m)|(salon de usos multiples)/gi', re.IGNORECASE)

ds_propiedades['sum'] = ds_propiedades['property_description'].apply(lambda x: 1 if regexp.search(x) else 0)

In [27]:
cantidad_sum = ds_propiedades[ds_propiedades['sum'] == 1].shape[0]
print(f"Vemos que la cantidad de propiedades con SUM es de {cantidad_sum}")

Vemos que la cantidad de propiedades con SUM es de 14847


#### Expensas

Vamos a buscar generar un parametro expensas_tipo que sea baja, mediana, alta o no aplica dependiendo del caso, para esto primero vamos a buscar los precios que se encuentran en el dataset y con eso hacer un rango de precios para cada tipo de expensas. Luego esto lo combinamos con otra busqueda en las descripciones para encontrar palabras que nos indiquen si es un precio alto, medio o bajo.

In [28]:
regexp = re.compile(r'expensas[\$0-9 \.,:]+|/gi', re.IGNORECASE)

ds_propiedades['expensas_price'] = ds_propiedades['property_description'].apply(lambda x: regexp.search(x).group() if regexp.search(x) else -1)

ds_propiedades['expensas_price'] = ds_propiedades['expensas_price'].apply(lambda x: x.lower().replace('expensas:','').replace('$','').replace(' ','').replace('.','').replace(',','.') if x != -1 else -1)

ds_propiedades['expensas_price'] =   ds_propiedades['expensas_price'].apply(lambda x: float(x) if str(x).isnumeric() else -1)

In [29]:
propiedades_con_precio_expensas = ds_propiedades[ds_propiedades['expensas_price'] > -1].shape[0]

print(f"Vemos que la cantidad de propiedades con expensas es de {propiedades_con_precio_expensas}")

Vemos que la cantidad de propiedades con expensas es de 4484


Expensas_tipo

In [30]:
regexp_sin = re.compile(r'sin expensas', re.IGNORECASE)
regexp_tipo = re.compile(r'expensas baja|media|alta|accesible|barata', re.IGNORECASE)

ds_propiedades['expensas_tipo'] = ds_propiedades['property_description'].apply(lambda x: 
  regexp_sin.search(x.lower()).group() if regexp_sin.search(x) else (regexp_tipo.search(x.lower()).group() if regexp_tipo.search(x) else 'no aplica')
)

ds_propiedades['expensas_tipo'] = ds_propiedades['expensas_tipo'].apply(lambda x: 'sin' if 'sin' in str(x) else x)
ds_propiedades['expensas_tipo'] = ds_propiedades['expensas_tipo'].apply(lambda x: 'bajas' if 'baja' in str(x) else x)
ds_propiedades['expensas_tipo'] = ds_propiedades['expensas_tipo'].apply(lambda x: 'bajas' if 'accesible' in str(x) else x)
ds_propiedades['expensas_tipo'] = ds_propiedades['expensas_tipo'].apply(lambda x: 'bajas' if 'barata' in str(x) else x)
ds_propiedades['expensas_tipo'] = ds_propiedades['expensas_tipo'].apply(lambda x: 'medias' if 'media' in str(x) else x)
ds_propiedades['expensas_tipo'] = ds_propiedades['expensas_tipo'].apply(lambda x: 'altas' if 'alta' in str(x) else x)

ds_propiedades['expensas_tipo'].value_counts()

no aplica    47149
medias       15184
bajas        11582
altas         9088
sin           2690
Name: expensas_tipo, dtype: int64

Vemos cuantas expensas categorizadas como 'no aplica' tienen un precio y las categorizamos acordemente

In [31]:
ds_propiedades_completar_tipo = ds_propiedades[(ds_propiedades['expensas_tipo'] == 'no aplica') & (ds_propiedades['expensas_price'] > 0)].copy()

if len(ds_propiedades_completar_tipo) > 0: ds_propiedades_completar_tipo['expensas_tipo'] = pd.qcut(ds_propiedades_completar_tipo['expensas_price'], q=3, labels=['bajas', 'medias', 'altas'])

ds_propiedades.loc[(ds_propiedades['expensas_tipo'] == 'no aplica') & (ds_propiedades['expensas_price'] > 0), 'expensas_tipo'] = ds_propiedades_completar_tipo['expensas_tipo']
ds_propiedades.drop(columns=['expensas_price'], inplace=True)

ds_propiedades['expensas_tipo'].value_counts()

no aplica    44344
medias       16106
bajas        12542
altas        10011
sin           2690
Name: expensas_tipo, dtype: int64

## Filtramos las columnas generadas

In [32]:
ds_propiedades.head()

,id,latitud,longitud,place_l3,property_type,property_rooms,property_bedrooms,property_surface_total,property_surface_covered,property_price,...,cochera,balcon,hidromasaje,jardin,gimnasio,a_estrenar,sotano,quincho,sum,expensas_tipo
0,F5IvmD1uzzdIR7b96ozt4g==,-34.558855,-58.452458,Nuñez,Departamento,4.0,3.0,160.0,146.0,460000.0,...,1,1,0,0,0,0,0,0,0,no aplica
1,IiT/P7WfYDcqbhu7tf5V4Q==,-34.582403,-58.443841,Colegiales,Departamento,2.0,1.0,50.0,46.0,148000.0,...,1,1,0,0,0,0,0,0,0,no aplica
2,6XXcElx/4OwCcH4NI06Oiw==,-34.605825,-58.422214,Almagro,Departamento,3.0,2.0,47.0,47.0,89000.0,...,0,0,0,0,0,0,0,0,0,no aplica
3,8R7hSzqWFk9rX4wBXxfORA==,-34.561971,-58.455670,Belgrano,Departamento,2.0,1.0,52.0,48.0,127000.0,...,0,1,0,0,0,1,0,0,0,medias
4,UAKYqQx9s0MfIdLzrJv8Ig==,-34.606738,-58.389426,Tribunales,Departamento,3.0,2.0,76.0,75.0,129900.0,...,0,1,0,0,0,0,0,0,0,no aplica


Ahora ya tenemos creadas las nuevas columnas, vamos a ver si podemos sacar algo de informacion de ellas

In [33]:
columnas_agregadas = ['piscina', 'cochera', 'balcon', 'hidromasaje', 'jardin', 'gimnasio', 'a_estrenar', 'sotano', 'quincho', 'sum', 'expensas_tipo']

ds_propiedades[columnas_agregadas].sum()

piscina                                                       9729
cochera                                                      27031
balcon                                                       52839
hidromasaje                                                   4047
jardin                                                        2688
gimnasio                                                      6601
a_estrenar                                                    8311
sotano                                                          83
quincho                                                       3173
sum                                                          14847
expensas_tipo    no aplicano aplicano aplicamediasno aplicamedi...
dtype: object

Podemos ver que Piscina, Cochera, sum y Balcon tienen una gran cantidad de datos por lo que dejarlos realmente puede aportar valor, ahora vamos a analizar los otros uno por uno

#### Jardin

Siendo que tenemos tan pocos datos no podemos sacar conclusiones acertadas de esto. El jardín representa la superficie descubierta de nuestra propiedad, que ya está en el dataset, por lo tanto sería información redundante, entonces lo sacamos

#### Hidromasaje, Gimnasio y Quincho

En estos casos tambien vemos que hay pocos, pero con el dominio del problema podemos asumir que son un valor agregado poco frecuente y que aunque tengamos pocos, es muy probable que esten bien y tengan una relacion directa con lo 'premium' que es la propiedad

#### A estrenar

Este dato tambien es poco frecuente, pero es un gran valor agregado a la propiedad y una de las cosas que mas se tienen en cuenta a la hora de comprar propiedades. Por esto lo dejamos sin lugar a dudas

#### Sotano 

El sotano es un dato que aparece en muy pocas propiedades, e incluso es un poco ambigüo ya que no tiene una descripcion detallada del mismo (puede ser una baulera o una planta completa). Por lo que tambien lo vamos a sacar

#### Sacamos las columnas que no mantenemos

In [34]:
ds_propiedades.drop(columns=['jardin', 'sotano'], inplace=True)

## Dataset final

In [35]:
ds_propiedades_xgboost = ds_propiedades.copy()

In [36]:
ds_propiedades_xgboost.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85693 entries, 0 to 85692
Data columns (total 23 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   id                        85693 non-null  object 
 1   latitud                   85693 non-null  float64
 2   longitud                  85693 non-null  float64
 3   place_l3                  85693 non-null  object 
 4   property_type             85693 non-null  object 
 5   property_rooms            85693 non-null  float64
 6   property_bedrooms         85693 non-null  float64
 7   property_surface_total    85693 non-null  float64
 8   property_surface_covered  85693 non-null  float64
 9   property_price            85693 non-null  float64
 10  property_title            85693 non-null  object 
 11  pxm2                      85693 non-null  float64
 12  tipo_precio               85693 non-null  object 
 13  property_description      85693 non-null  object 
 14  piscin

## 1b) Entranamiento XGBoost

In [37]:
ds_propiedades_xgboost.head()

,id,latitud,longitud,place_l3,property_type,property_rooms,property_bedrooms,property_surface_total,property_surface_covered,property_price,...,property_description,piscina,cochera,balcon,hidromasaje,gimnasio,a_estrenar,quincho,sum,expensas_tipo
0,F5IvmD1uzzdIR7b96ozt4g==,-34.558855,-58.452458,Nuñez,Departamento,4.0,3.0,160.0,146.0,460000.0,...,Hermoso piso de 4 ambientes con dependencia y ...,0,1,1,0,0,0,0,0,no aplica
1,IiT/P7WfYDcqbhu7tf5V4Q==,-34.582403,-58.443841,Colegiales,Departamento,2.0,1.0,50.0,46.0,148000.0,...,2 ambientes con balcón y vista totalmente abie...,0,1,1,0,0,0,0,0,no aplica
2,6XXcElx/4OwCcH4NI06Oiw==,-34.605825,-58.422214,Almagro,Departamento,3.0,2.0,47.0,47.0,89000.0,...,**DIAZ VELEZ 4291 Y PRINGLES** Almagro.<br><br...,0,0,0,0,0,0,0,0,no aplica
3,8R7hSzqWFk9rX4wBXxfORA==,-34.561971,-58.455670,Belgrano,Departamento,2.0,1.0,52.0,48.0,127000.0,...,IMPECABLE 2 AMBIENTES A ESTRENAR!<...,0,0,1,0,0,1,0,0,medias
4,UAKYqQx9s0MfIdLzrJv8Ig==,-34.606738,-58.389426,Tribunales,Departamento,3.0,2.0,76.0,75.0,129900.0,...,"Excelente piso antiguo, lindísimo por sus tech...",0,0,1,0,0,0,0,0,no aplica


Separamos en columnas dummies para property_type, place_l3 y expensas_tipo

In [38]:
ds_propiedades_xgboost = pd.get_dummies(ds_propiedades_xgboost, columns=['property_type', 'place_l3', 'expensas_tipo'])

In [39]:
ds_propiedades_xgboost.drop(columns=['property_description', 'property_title', 'id', 'tipo_precio', 'pxm2'], inplace=True)

In [40]:
ds_propiedades_xgboost.head()

,latitud,longitud,property_rooms,property_bedrooms,property_surface_total,property_surface_covered,property_price,piscina,cochera,balcon,...,place_l3_Villa Riachuelo,place_l3_Villa Santa Rita,place_l3_Villa Soldati,place_l3_Villa Urquiza,place_l3_Villa del Parque,expensas_tipo_altas,expensas_tipo_bajas,expensas_tipo_medias,expensas_tipo_no aplica,expensas_tipo_sin
0,-34.558855,-58.452458,4.0,3.0,160.0,146.0,460000.0,0,1,1,...,0,0,0,0,0,0,0,0,1,0
1,-34.582403,-58.443841,2.0,1.0,50.0,46.0,148000.0,0,1,1,...,0,0,0,0,0,0,0,0,1,0
2,-34.605825,-58.422214,3.0,2.0,47.0,47.0,89000.0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,-34.561971,-58.455670,2.0,1.0,52.0,48.0,127000.0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
4,-34.606738,-58.389426,3.0,2.0,76.0,75.0,129900.0,0,0,1,...,0,0,0,0,0,0,0,0,1,0


In [41]:
from sklearn.model_selection import train_test_split
from sklearn import metrics
import xgboost as xgb

x_train, x_test, y_train, y_test = train_test_split(ds_propiedades_xgboost, ds_propiedades_xgboost['property_price'], test_size=0.20, random_state=1)
x_train.drop(columns=['property_price'], inplace=True)
x_test.drop(columns=['property_price'], inplace=True)

### Modelo inicial sin optimizar (con los hiperparametros del TP1)

En el tp1 hicimos una optimizacion de hiperparametros con StratifiedKFold pero no logramos mejoria, por lo que los hiperparametros son los default del modelo

In [42]:
xgb_model = xgb.XGBRegressor(objective = 'reg:squarederror', random_state=1)

In [43]:
xgb_model.fit(x_train, y_train)

y_pred = xgb_model.predict(x_test)

In [44]:
performance = pd.DataFrame({'Valor Real': y_test,
                            'Prediccion': y_pred,
                            'Error': y_test - y_pred})

performance.head()

,Valor Real,Prediccion,Error
11387,170000.0,183790.640625,-13790.640625
78283,360000.0,301092.062500,58907.937500
15574,280000.0,282107.937500,-2107.937500
17122,129000.0,146419.031250,-17419.031250
8069,260000.0,266423.500000,-6423.500000


In [45]:
mse = metrics.mean_squared_error(
        y_true  = y_test,
        y_pred  = y_pred,
        squared = True
         )

print(f"El error (mse) de test es: {mse}")

rmse = metrics.mean_squared_error(
        y_true  = y_test,
        y_pred  = y_pred,
        squared = False
         )

print(f"El error (rmse) de test es: {rmse}")  
print(f"El error (rmse) de test relativo es: {rmse / y_test.mean() * 100}%")

El error (mse) de test es: 1473587857.3848019
El error (rmse) de test es: 38387.33980604546
El error (rmse) de test relativo es: 22.62140499622676%


In [46]:
score_xgb_train = xgb_model.score(x_train, y_train)
score_xgb_test = xgb_model.score(x_test, y_test)
print(f"El coeficiente de Determinación R2 en train es: {score_xgb_train}")
print(f"El coeficiente de Determinación R2 en test es: {score_xgb_test}")

El coeficiente de Determinación R2 en train es: 0.9232276232123919
El coeficiente de Determinación R2 en test es: 0.89301154705351


Viendo la comparacion directa contra el mismo modelo en el TP1, podemos ver que el modelo mejoro bastante con los nuevos datos que generamos.

El error (mse) de test pasa de 1907948928.924305 -> 1473587857.3848019

El error (rmse) de test pasa de 43680.074735791204 -> 38387.33980604546  (12,1% mejor)

El R2 en train pasa de 0.8918109411819658 -> 0.9232276232123919 

El R2 en test pasa de 0.860060265420469 -> 0.89301154705351


Esto demuestra que las variables que generamos a partir de las descripciones son muy utiles para el modelo, y que el modelo mejora bastante con ellas.

### Probamos optimizar los hiperparametros

In [47]:
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import make_scorer, r2_score


xgb_cv = xgb.XGBRegressor(objective = 'reg:squarederror', random_state=2)

n=10

#xgb param grid
xgb_param_grid = {
    'n_estimators': [50, 100],
    'learning_rate': [0.1, 0.3],
    'max_depth': [3, 7],
    'gamma': [0.3, 0.4],
}

folds=5
kfoldcv = StratifiedKFold(n_splits=folds, random_state=10,shuffle=True)

scorer_fn = make_scorer(r2_score)

random_search_cv = RandomizedSearchCV(estimator=xgb_cv,
                              param_distributions = xgb_param_grid,
                              scoring='neg_mean_squared_error',
                              cv=kfoldcv,
                              n_iter=n,
                              random_state=10) 

random_fit = random_search_cv.fit(x_train[['property_rooms']], y_train)

c:\Users\f1203\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\model_selection\_split.py:684: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


In [48]:
print(random_fit.best_params_)

{'n_estimators': 100, 'max_depth': 7, 'learning_rate': 0.1, 'gamma': 0.3}


In [49]:
parametros = random_fit.best_params_

xgb_model = xgb.XGBRegressor(learning_rate=parametros['learning_rate'], n_estimators=parametros['n_estimators'], gamma=parametros['gamma'], max_depth = parametros['max_depth'], random_state=1)

model = xgb_model.fit(X = x_train, y = y_train) 

y_pred = model.predict(x_test)

In [50]:
performance = pd.DataFrame({'Valor Real': y_test,
                            'Prediccion': y_pred,
                            'Error': y_test - y_pred})


performance.head()

,Valor Real,Prediccion,Error
11387,170000.0,170586.296875,-586.296875
78283,360000.0,284469.843750,75530.156250
15574,280000.0,264595.593750,15404.406250
17122,129000.0,145836.109375,-16836.109375
8069,260000.0,258944.703125,1055.296875


In [51]:
mse = metrics.mean_squared_error(
        y_true  = y_test,
        y_pred  = y_pred,
        squared = True
         )

print(f"El error (mse) de test es: {mse}")

rmse = metrics.mean_squared_error(
        y_true  = y_test,
        y_pred  = y_pred,
        squared = False
         )

print(f"El error (rmse) de test es: {rmse}")  
print(f"El error (rmse) de test relativo es: {rmse / y_test.mean() * 100}%")

El error (mse) de test es: 1515631827.2171123
El error (rmse) de test es: 38931.11643938705
El error (rmse) de test relativo es: 22.941848963234023%


In [52]:
#Coef. de Determinación R2
score_xgb_train = xgb_model.score(x_train, y_train)
score_xgb_test = xgb_model.score(x_test, y_test)
print(f"El coeficiente de Determinación R2 en train es: {score_xgb_train}")
print(f"El coeficiente de Determinación R2 en test es: {score_xgb_test}")

El coeficiente de Determinación R2 en train es: 0.9123206582983798
El coeficiente de Determinación R2 en test es: 0.8899589843810198


La busqueda de hiperparametros encuentra una configuracion con un resultado similar al modelo inicial, pero con un tiempo de entrenamiento mucho mayor. Por esto mantenemos el modelo inicial.

# <center> Redes Neuronales <center/> 

## Clasificacion

In [83]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import f1_score,  recall_score, precision_score, accuracy_score
from sklearn.preprocessing import StandardScaler

import tensorflow as tf
from tensorflow import keras
from keras.utils.vis_utils import plot_model

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [84]:
ds_propiedades_para_clasificacion = ds_propiedades.copy()

ds_propiedades_para_clasificacion.head()

,id,latitud,longitud,place_l3,property_type,property_rooms,property_bedrooms,property_surface_total,property_surface_covered,property_price,...,property_description,piscina,cochera,balcon,hidromasaje,gimnasio,a_estrenar,quincho,sum,expensas_tipo
0,F5IvmD1uzzdIR7b96ozt4g==,-34.558855,-58.452458,Nuñez,Departamento,4.0,3.0,160.0,146.0,460000.0,...,Hermoso piso de 4 ambientes con dependencia y ...,0,1,1,0,0,0,0,0,no aplica
1,IiT/P7WfYDcqbhu7tf5V4Q==,-34.582403,-58.443841,Colegiales,Departamento,2.0,1.0,50.0,46.0,148000.0,...,2 ambientes con balcón y vista totalmente abie...,0,1,1,0,0,0,0,0,no aplica
2,6XXcElx/4OwCcH4NI06Oiw==,-34.605825,-58.422214,Almagro,Departamento,3.0,2.0,47.0,47.0,89000.0,...,**DIAZ VELEZ 4291 Y PRINGLES** Almagro.<br><br...,0,0,0,0,0,0,0,0,no aplica
3,8R7hSzqWFk9rX4wBXxfORA==,-34.561971,-58.455670,Belgrano,Departamento,2.0,1.0,52.0,48.0,127000.0,...,IMPECABLE 2 AMBIENTES A ESTRENAR!<...,0,0,1,0,0,1,0,0,medias
4,UAKYqQx9s0MfIdLzrJv8Ig==,-34.606738,-58.389426,Tribunales,Departamento,3.0,2.0,76.0,75.0,129900.0,...,"Excelente piso antiguo, lindísimo por sus tech...",0,0,1,0,0,0,0,0,no aplica


Modificamos el dataframe para poder usarlo en la red neuronal. Nos quedamos con las columnas necesarias, normalizamos y modificamos los tipos de datos 

In [85]:
# Eliminamos estas columnas porque no las necesitamos para el modelo
ds_propiedades_para_clasificacion.drop(['property_title', 'id','property_description'], axis=1, inplace=True)

#Eliminamos las columnas que tienen info del precio
ds_propiedades_para_clasificacion.drop(['property_price', 'pxm2'], axis=1, inplace=True)

# One hot encoding de las columnas categóricas
ds_propiedades_para_clasificacion = pd.get_dummies(ds_propiedades_para_clasificacion, columns=["place_l3", "property_type"], dtype=np.int8)

In [86]:
# Convertimos tipo_precio y expensas_tipo a una columna numerica
ds_propiedades_para_clasificacion.tipo_precio.replace({'bajo':0,'medio':1,'alto':2}, inplace=True)
ds_propiedades_para_clasificacion.expensas_tipo.replace({'sin': -2, 'no aplica':-1,'bajas':0,'medias':1,'altas':2}, inplace=True)

In [87]:
# Seleccionamos las columnas que vamos a usar para el modelo
columnas_predictoras = ds_propiedades_para_clasificacion.columns.to_list()
columnas_predictoras.remove('tipo_precio')
d_in=len(columnas_predictoras)

In [88]:
# Normalizamos las variables de entrada
scaler = MinMaxScaler()
ds_propiedades_para_clasificacion[columnas_predictoras] = scaler.fit_transform(ds_propiedades_para_clasificacion[columnas_predictoras])
ds_propiedades_para_clasificacion

ds_propiedades_para_clasificacion.astype('float32', copy=False)

ds_propiedades_para_clasificacion.head()

,latitud,longitud,property_rooms,property_bedrooms,property_surface_total,property_surface_covered,tipo_precio,piscina,cochera,balcon,...,place_l3_Villa Pueyrredón,place_l3_Villa Real,place_l3_Villa Riachuelo,place_l3_Villa Santa Rita,place_l3_Villa Soldati,place_l3_Villa Urquiza,place_l3_Villa del Parque,property_type_Casa,property_type_Departamento,property_type_PH
0,0.859587,0.416415,0.500000,0.6,0.477477,0.472313,2,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.715143,0.462460,0.166667,0.2,0.147147,0.146580,2,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.571472,0.578015,0.333333,0.4,0.138138,0.149837,1,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.840478,0.399254,0.166667,0.2,0.153153,0.153094,1,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.565870,0.753212,0.333333,0.4,0.225225,0.241042,0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [89]:
# Separamos en train y test
x_train, x_test, y_train, y_test = train_test_split(ds_propiedades_para_clasificacion.loc[:,columnas_predictoras],ds_propiedades_para_clasificacion.loc[:,'tipo_precio'], test_size=0.20, random_state=1)

Creamos la red neuronal

Ya sabemos que las clases son 3, igualmente las calculamos?

In [97]:
# calcula la cantidad de clases posibles para clasificar
classes = int(ds_propiedades_para_clasificacion.loc[:,'tipo_precio'].max()+1)


modelo = keras.Sequential([
    # input_shape solo en la primer capa
    # Capa con 3 salidas, activación relu
    keras.layers.Dense(5,input_shape=(d_in,), activation='relu'),

    # keras.layers.Dense(5, activation='tanh'),
    # keras.layers.Dense(5, activation='relu'),
    keras.layers.Dense(5, activation='sigmoid'),

    #la ultima capa si o si tiene que tener tantas salidas como clases, y softmax
    keras.layers.Dense(classes, activation='softmax')])

modelo.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_30 (Dense)            (None, 5)                 380       
                                                                 
 dense_31 (Dense)            (None, 5)                 30        
                                                                 
 dense_32 (Dense)            (None, 3)                 18        
                                                                 
Total params: 428
Trainable params: 428
Non-trainable params: 0
_________________________________________________________________


In [98]:
# Compilamos el modelo
modelo.compile(
  optimizer=keras.optimizers.SGD(learning_rate=0.1), 
  loss='sparse_categorical_crossentropy', 
  metrics=['accuracy'], 
)

# Entrenamiento del modelo
history = modelo.fit(x_train,y_train,epochs=150,batch_size=200,verbose=True)

Epoch 1/150
343/343 [==============================] - 3s 6ms/step - loss: 1.0172 - accuracy: 0.4945
Epoch 2/150
343/343 [==============================] - 2s 6ms/step - loss: 0.9001 - accuracy: 0.5480
Epoch 3/150
343/343 [==============================] - 1s 4ms/step - loss: 0.8131 - accuracy: 0.6199
Epoch 4/150
343/343 [==============================] - 1s 3ms/step - loss: 0.7838 - accuracy: 0.6383
Epoch 5/150
343/343 [==============================] - 1s 3ms/step - loss: 0.7714 - accuracy: 0.6450
Epoch 6/150
343/343 [==============================] - 1s 3ms/step - loss: 0.7637 - accuracy: 0.6476
Epoch 7/150
343/343 [==============================] - 1s 3ms/step - loss: 0.7572 - accuracy: 0.6491
Epoch 8/150
343/343 [==============================] - 1s 3ms/step - loss: 0.7521 - accuracy: 0.6511
Epoch 9/150
343/343 [==============================] - 1s 3ms/step - loss: 0.7478 - accuracy: 0.6534
Epoch 10/150
343/343 [==============================] - 1s 3ms/step - loss: 0.7443 - accura

Vemos las metricas obtenidas

In [99]:
y_pred = modelo.predict(x_test)

y_pred = np.argmax(y_pred,axis = 1) 

print("  Accuracy : %.3f" % accuracy_score(y_test, y_pred))
print("  Precision: %.3f" % precision_score(y_test, y_pred, average='macro'))
print("     Recall: %.3f" % recall_score(y_test, y_pred, average='macro'))
print("  f-measure: %.3f" % f1_score(y_test, y_pred, average='macro'))

536/536 [==============================] - 4s 7ms/step
  Accuracy : 0.681
  Precision: 0.708
     Recall: 0.639
  f-measure: 0.660


Las metricas no son malas, pero vamos a ver si podemos optimizar aun mas.
Para buscar otras metricas, vamos a hacerlo iterando por distintas posibilades para los hiperparametros y quedarnos con la que nos de las mejores.

In [70]:
import random

random.seed(1)

activations = ['relu', 'tanh', 'sigmoid', 'softmax']


mejores_hyperparametros = {
    'func_activacion_1': '',
    'func_activacion_2': '',
    'func_activacion_3': '',
    'epochs': 0,
    'batch_size': 0,
}

mejores_metricas = {
  'prom' : 0,
  'precision' : 0,
  'recall' : 0,
  'f-measure' : 0
}

for i in range(0,5):

  cant_salidas = random.randint(0,4)
  func_activacion_1 = activations[random.randint(0,3)]
  func_activacion_2 = activations[random.randint(0,3)]
  func_activacion_3 = activations[random.randint(0,3)]
  modelo = keras.Sequential([
      # input_shape solo en la primer capa
      # Capa con 3 salidas, activación relu
      keras.layers.Dense(3,input_shape=(d_in,), activation=activations[random.randint(0,3)]),

      keras.layers.Dense(cant_salidas, activation=func_activacion_1),
      keras.layers.Dense(cant_salidas, activation=func_activacion_2),
      keras.layers.Dense(cant_salidas, activation=func_activacion_3),

      #la ultima capa si o si tiene que tener tantas salidas como clases, y softmax 
      keras.layers.Dense(classes, activation='softmax')])

  # Compilamos el modelo
  modelo.compile(
    optimizer=keras.optimizers.SGD(learning_rate=0.1), 
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'], 
  )
  
  # Entrenamiento del modelo
  epochs = random.randint(50,150)
  batch_size = random.randint(50,200)


  print(f"Probando con: {cant_salidas} salidas, {func_activacion_1}, {func_activacion_2}, {func_activacion_3}, {epochs} epochs, {batch_size} batch_size")
  history = modelo.fit(x_train,y_train,epochs=epochs,batch_size=batch_size,verbose=False)

  y_pred = modelo.predict(x_test)
  y_pred = np.argmax(y_pred,axis = 1) 

  # Utilizamos las metricas que pide la consigna
  pres = precision_score(y_test, y_pred, average='macro')
  recall = recall_score(y_test, y_pred, average='macro')
  f1 = f1_score(y_test, y_pred, average='macro')

  # Calculamos la media de las metricas
  metricas = (pres + recall + f1) / 3
  
  print("Metricas obtenidas:")
  print(f"precision:{pres}")
  print(f"recall:{recall}")
  print(f"f1:{f1}")
  print()

  if metricas > mejores_metricas['prom']:
    mejores_metricas['prom'] = metricas
    
    mejores_metricas['precision'] = pres
    mejores_metricas['recall'] = recall
    mejores_metricas['f-measure'] = f1

    mejores_hyperparametros['cant_salidas'] = cant_salidas
    mejores_hyperparametros['func_activacion_1'] = func_activacion_1
    mejores_hyperparametros['func_activacion_2'] = func_activacion_2
    mejores_hyperparametros['func_activacion_3'] = func_activacion_3
    mejores_hyperparametros['epochs'] = epochs
    mejores_hyperparametros['batch_size'] = batch_size
  

print(mejores_metricas)
print(mejores_hyperparametros)

Probando con: 1 salidas, relu, sigmoid, relu, 150 epochs, 103 batch_size
536/536 [==============================] - 1s 2ms/step


c:\Users\f1203\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Metricas obtenidas:
precision:0.16603458000272284
recall:0.3333333333333333
f1:0.2216596562289038

Probando con: 0 salidas, softmax, relu, softmax, 107 epochs, 118 batch_size
536/536 [==============================] - 2s 3ms/step


c:\Users\f1203\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Metricas obtenidas:
precision:0.08434953420075071
recall:0.3333333333333333
f1:0.13463090581734646

Probando con: 1 salidas, relu, sigmoid, relu, 98 epochs, 105 batch_size


KeyboardInterrupt: 

Buscando de esta forma, las mejores metricas obtenidas son iguales que las obtenidas en el punto anterior, pero con y menos salidas por capa.


In [ ]:
print("Mejores Metricas obtenidas:")
print(f"precision:{mejores_metricas['precision']}")
print(f"recall:{mejores_metricas['recall']}")
print(f"f1:{mejores_metricas['f-measure']}")

Mejores Metricas obtenidas:
precision:0.7064291108803388
recall:0.6223626534146017
f1:0.64707483534526


## Regresion

Usamos esta funcion para graficar mas comodo

In [ ]:
def plot_regression(modelo,x,y,title="",xlabel="x",ylabel="y"):
    plt.figure()
    
    plt.plot(x,y,"o",label="Valores verdaderos")
    plt.plot(x,modelo.predict(x),"x",label="Valores estimados")
    
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.title(title)
    plt.legend()
    plt.show()

In [ ]:
ds_propiedades_para_regresion = ds_propiedades.copy()

Transformamos los datos del dataframe

In [ ]:
# Eliminamos estas columnas porque no las necesitamos para el modelo
ds_propiedades_para_regresion.drop(['property_title', 'id', 'property_description'], axis=1, inplace=True)

#Eliminamos las columnas que tienen info del precio
ds_propiedades_para_regresion.drop(['tipo_precio','pxm2'], axis=1, inplace=True)

# One hot encoding de las columnas categóricas
ds_propiedades_para_regresion = pd.get_dummies(ds_propiedades_para_regresion, columns=["place_l3", "property_type"], dtype=np.int8)

# Seleccionamos las columnas que vamos a usar para el modelo
columnas_predictoras=ds_propiedades_para_regresion.columns.to_list()

# Convertimos expensas_tipo a una columna numerica
ds_propiedades_para_regresion.expensas_tipo.replace({'sin': -2, 'no aplica':-1,'bajas':0,'medias':1,'altas':2}, inplace=True)

# Normalizamos las variables de entrada
scaler = MinMaxScaler()
ds_propiedades_para_regresion[columnas_predictoras] = scaler.fit_transform(ds_propiedades_para_regresion[columnas_predictoras])

# Eliminamos property_price porque es la variable que queremos predecir
columnas_predictoras.remove('property_price')

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(ds_propiedades_para_regresion.loc[:,columnas_predictoras],ds_propiedades_para_regresion.loc[:,'property_price'],test_size=0.2)

sscaler=StandardScaler()
sscaler.fit(pd.DataFrame(x_train))

x_train_transform=sscaler.transform(pd.DataFrame(x_train))
x_test_transform=sscaler.transform(pd.DataFrame(x_test))

ds_propiedades_para_regresion

,latitud,longitud,property_rooms,property_bedrooms,property_surface_total,property_surface_covered,property_price,piscina,cochera,balcon,...,place_l3_Villa Pueyrredón,place_l3_Villa Real,place_l3_Villa Riachuelo,place_l3_Villa Santa Rita,place_l3_Villa Soldati,place_l3_Villa Urquiza,place_l3_Villa del Parque,property_type_Casa,property_type_Departamento,property_type_PH
0,0.859587,0.416415,0.500000,0.6,0.477477,0.472313,0.383966,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.715143,0.462460,0.166667,0.2,0.147147,0.146580,0.120675,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.571472,0.578015,0.333333,0.4,0.138138,0.149837,0.070886,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.840478,0.399254,0.166667,0.2,0.153153,0.153094,0.102954,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.565870,0.753212,0.333333,0.4,0.225225,0.241042,0.105401,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85688,0.728453,0.519028,0.333333,0.4,0.171171,0.185668,0.143460,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
85689,0.465723,0.493996,0.833333,0.8,0.549550,0.596091,0.350211,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
85690,0.407469,0.645797,0.166667,0.2,0.132132,0.130293,0.066667,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
85691,0.687028,0.098150,0.166667,0.2,0.255255,0.143322,0.097046,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [ ]:
# Creo un modelo Red Neuronal
d_in=len(columnas_predictoras)
d_out=1

modelo1 = keras.Sequential([
    keras.layers.Dense(1,input_shape=(d_in,)), #, activation="relu"

    keras.layers.Dense(3, activation='tanh'),
    keras.layers.Dense(3, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
    ])
modelo1.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_30 (Dense)            (None, 1)                 76        
                                                                 
 dense_31 (Dense)            (None, 3)                 6         
                                                                 
 dense_32 (Dense)            (None, 3)                 12        
                                                                 
 dense_33 (Dense)            (None, 1)                 4         
                                                                 
Total params: 98
Trainable params: 98
Non-trainable params: 0
_________________________________________________________________


In [ ]:
plot_regression(modelo1,x_train_transform,y_train,title=f"Modelo inicial",xlabel=variable_predictora,ylabel="tipo_precio")

NameError: name 'variable_predictora' is not defined

In [ ]:
modelo1.compile(
  optimizer=keras.optimizers.SGD(learning_rate=0.001), 
  loss='mse', 
  metrics=['mae'], 
)

In [ ]:
y_pred = modelo1.fit(x_train_transform,y_train,epochs=50,batch_size=85,verbose=True)

Epoch 1/50
807/807 [==============================] - 1s 635us/step - loss: 0.1391 - mae: 0.3531
Epoch 2/50
807/807 [==============================] - 1s 620us/step - loss: 0.1011 - mae: 0.2950
Epoch 3/50
807/807 [==============================] - 0s 617us/step - loss: 0.0764 - mae: 0.2513
Epoch 4/50
807/807 [==============================] - 1s 619us/step - loss: 0.0601 - mae: 0.2184
Epoch 5/50
807/807 [==============================] - 1s 629us/step - loss: 0.0489 - mae: 0.1933
Epoch 6/50
807/807 [==============================] - 1s 634us/step - loss: 0.0409 - mae: 0.1739
Epoch 7/50
807/807 [==============================] - 0s 616us/step - loss: 0.0351 - mae: 0.1585
Epoch 8/50
807/807 [==============================] - 0s 607us/step - loss: 0.0308 - mae: 0.1463
Epoch 9/50
807/807 [==============================] - 0s 603us/step - loss: 0.0274 - mae: 0.1363
Epoch 10/50
807/807 [==============================] - 0s 607us/step - loss: 0.0247 - mae: 0.1280
Epoch 11/50
807/807 [========

In [ ]:
y_pred=modelo1.predict(x_test_transform)
y_pred = y_pred.flatten()
performance = pd.DataFrame({'Valor Real': y_test,
                            'Prediccion': y_pred,
                            'Error': y_test - y_pred})


performance.head()

536/536 [==============================] - 0s 471us/step


,Valor Real,Prediccion,Error
40755,0.067511,0.254246,-0.186736
80532,0.109705,0.153817,-0.044113
51314,0.210970,0.127455,0.083516
85116,0.345992,0.124392,0.221600
71613,0.050211,0.096458,-0.046248


In [ ]:
from sklearn import metrics

mse = metrics.mean_squared_error(
        y_true  = y_test,
        y_pred  = y_pred,
        squared = True
         )

print(f"El error (mse) de test es: {mse}")



rmse = metrics.mean_squared_error(
        y_true  = y_test,
        y_pred  = y_pred,
        squared = False
         )

print(f"El error (rmse) de test es: {rmse}")    

El error (mse) de test es: 0.007901013480400742
El error (rmse) de test es: 0.08888764526299896


In [ ]:
plot_regression(modelo1,x_train_transform,y_train,title=f"Modelo Final",xlabel=variable_predictora,ylabel="MPG")

In [ ]:
plot_regression(modelo1,x_test_transform,y_test,title=f"Modelo Final",xlabel=variable_predictora,ylabel="MPG")